In [1]:
import sys
import os
import pandas as pd
import numpy as np
import datetime as dt
import h5py
import json
import requests
import base64
sys.path.append('../')

pd.options.display.max_rows=99

In [2]:
github_userName = 'Tanag3r'
ebird_token = 'j6c7l80ga2ib'

In [3]:
##pull in trailhead stops
trailheadRef = pd.DataFrame()
trailheadRef_json = 'https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadsRef.json'
response = requests.get(trailheadRef_json)
trailheadRef = pd.DataFrame(response.json())
##trailheadRef.set_index('StopName', inplace=True

trailheadRef.convert_dtypes()

Route         object
StopName      object
Address       object
Latitude     float64
Longitude    float64
dtype: object

In [4]:
##set types for error checking, other cleaning
trailheadRef['Latitude'] = trailheadRef['Latitude'].fillna(0.00)
trailheadRef['Longitude'] = trailheadRef['Longitude'].fillna(0.00)
trailheadRef['Latitude'] = trailheadRef['Latitude'].astype('float64',errors='raise')
trailheadRef['Longitude'] = trailheadRef['Longitude'].astype('float64',errors='raise')
trailheadRef['CoordinateTest'] = ''
trailheadRef['CoordinateTest'] = trailheadRef['CoordinateTest'].astype('str')
trailheadRef['AddressTest'] = ''
trailheadRef['AddressTest'] = trailheadRef['AddressTest'].astype('str')
trailheadRef['failCode'] = ''
trailheadRef['failCode'] = trailheadRef['failCode'].astype('str')

##todo #10 finish this evaluator engine

##warnings:
trailheadRef.loc[(abs(trailheadRef['Latitude']) < 1) | (abs(trailheadRef['Longitude']) < 1), 'CoordinateTest'] = 'GEO10'
trailheadRef.loc[(trailheadRef['Address'].isna()) | (trailheadRef['Address'].isnull()) | (trailheadRef['Address'] == ''), 'AddressTest'] = 'GEO20'
##failures:
trailheadRef.loc[(trailheadRef['CoordinateTest'] != '') & (trailheadRef['AddressTest'] != ''), 'failCode'] = 'GEO30'
##update the test below when the coordinate fetch engine is complete to reflect failure of fetch
trailheadRef.loc[(trailheadRef['CoordinateTest'] != '') & (trailheadRef['AddressTest'] == ''), 'failCode'] = 'GEO31'

##TODO: fetch coordinates of trailheads using addresses
##while trailheadRef['GeoTest'] is 'Fail':
    ##do the thing

##todo #9 write publisher for failed geoTest

trailheadRef_clean = pd.DataFrame()
trailheadRef_clean = trailheadRef[trailheadRef['failCode'] == '']
trailheadRef_clean.drop(columns=['CoordinateTest','AddressTest','failCode'],inplace=True,errors='ignore')


C:\Users\lukew\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [14]:
trailheadRef_clean.head()

,Route,StopName,Address,Latitude,Longitude
0,IssaquahAlps,EastSunsetWay,"661-831 E Sunset Way, Issaquah, WA 98027",47.529635,-122.025119
1,IssaquahAlps,HighSchool,"Parking lot, The Rainier Trail, Issaquah, WA 9...",47.519345,-122.029801
2,IssaquahAlps,ChiricoTrail_PooPooPoint,"11400 Issaquah-Hobart Road Southeast, Issaquah...",47.499949,-122.021730
3,IssaquahAlps,SquakMountain,"13201 Squak Mountain Rd SE, Issaquah, WA 98027",47.481465,-122.053997
5,IssaquahAlps,MargaretsWay,"190th Ave SE, Issaquah, WA 98027",47.506620,-122.086660


In [10]:
##fetch all hotspots within 4 kilometers of each trailhead

trailheadHotspots = []

for StopName in trailheadRef_clean.itertuples():
    url = '''https://api.ebird.org/v2/ref/hotspot/geo?lat={}&lng={}&dist=4&fmt=json'''.format(StopName.Latitude,StopName.Longitude)
    ebirdapi_auth_header = {'X-eBirdApiToken': ebird_token}
    ebird_request = requests.get(url,headers=ebirdapi_auth_header)
    ebird_response = pd.DataFrame(ebird_request.json())
    if ebird_request.status_code == requests.codes.ok:
        ebird_response['StopName'] = StopName.StopName
        trailheadHotspots.append(ebird_response)
    ebird_request.raise_for_status()

In [11]:
all_trailheadHotspots = pd.concat(trailheadHotspots)

In [13]:
##Be nice to eBirds and print the results
today = dt.datetime.today()
filename = 'trailheadHotspots_Y{}_M{}_D{}.csv'.format(today.year,today.month,today.day)

all_trailheadHotspots.to_csv(filename,sep=',',index=False)